<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/XMLS_FROM_GDRIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [2]:
# 
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
PHI_data_path = "/content/drive/My Drive/a_PROJECTS/PHI/PHI_data"

In [4]:
for inscription_number in inscription_numbers: # get inscription numbers from Petra's data
   try:
     # open the file
    except:
      # add it to a list of missing inscriptions...

OSError: ignored

In [0]:
len(filenames)

In [0]:
# based on this: https://mkyong.com/python/python-how-to-list-all-files-in-a-directory/
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(PHI_data_path):
    for file in f:
        if '.xml' in file:
            files.append(os.path.join(r, file))
for f in files:
    print(f)